In [1]:
import glob
import pandas as pd
import numpy as np
import os
def split_dataframe_to_chunks(df, n):
    df_len = len(df)
    count = 0
    dfs = []

    while True:
        if count > df_len-1:
            break

        start = count
        count += n
        dfs.append(df.iloc[start : count])
    return dfs

def delete_white_space(df):
    for i in df.columns:
        df[i] = df[i].astype(str).str.strip()
        df[i] = pd.to_numeric(df[i], downcast="float")
    return df

def FillMissingValues(DataFrame): #not fill out last columns
    DataFrame2 = DataFrame.iloc[:,:-1]
    print(DataFrame2.isna().sum().sum())
    DataFrame2 = DataFrame2.fillna(DataFrame2.mean())#full out the empty values by mean value of each column. 
    print(DataFrame2.isna().sum().sum())
    print(DataFrame.isna().sum().sum())
    DataFrame.iloc[:,:-1] = DataFrame2
    print(DataFrame.isna().sum().sum())
    return DataFrame

def Max_df(dfs,name): #dfs: list of df from split_dataframe_to_chunks; name: column name for check max value
    import warnings
    warnings.filterwarnings('ignore')
    newDF = pd.DataFrame()
    for df0 in dfs:
        df0 = df0.dropna()
        df0 = delete_white_space(df0)
        df0 = df0.dropna()
        if df0.empty==False:
            df0[name].astype(float)
            df0 = df0.reset_index(drop=True)
            maxrow = df0[name].idxmax()
            newDF = newDF.append(df0.iloc[[maxrow]])
    return newDF.reset_index(drop=True)

def Get_MaxAndARandomValue(dfs,name): 
    #dfs: list of df from split_dataframe_to_chunks; 
    #name: column name for check max value -> label 1 for peak concentration.
    #get ranrom value which is not max -> label 0 for not peak concentraion.
    import warnings
    warnings.filterwarnings('ignore')
    newDF = pd.DataFrame()
    np.random.seed(1234)
    for df0 in dfs:
        df0 = df0.dropna()
        df0 = delete_white_space(df0)
        df0 = df0.dropna()
        df0['Class']=np.array(np.zeros(len(df0),dtype=int)).tolist()
        if df0.empty==False:
            df0[name].astype(float)
            df0 = df0.reset_index(drop=True)
            maxrow = df0[name].idxmax()
            df0.at[maxrow, 'Class'] = 1
            newDF = newDF.append(df0.iloc[[maxrow]])
            #get a random row which is different from max. 
            r = np.random.randint(0,len(df0))
            if(len(df0)==1):
                continue
            else:
                while(r==maxrow):
                    r = np.random.randint(0,len(df0))
                newDF = newDF.append(df0.iloc[[r]])
    return newDF.reset_index(drop=True)


In [2]:
import glob
import pandas as pd
import numpy as np
import os

filenames = glob.glob("C:\data\PeakConcentration-Idea2-2-Clasification\data\*")


dest_dir_Morning = 'C:\data\PeakConcentration-Idea2-2-Clasification\Morning_Peak_Values\\'
dest_dir_Evening = 'C:\data\PeakConcentration-Idea2-2-Clasification\Evening_Peak_Values\\'
dest_dir_FullDay = 'C:\data\PeakConcentration-Idea2-2-Clasification\FullDay_Peak_Values\\'
def Write_To_File(dest_dir,df, filename):
    print(np.shape(df))
    training = df[df['Index']<=35064] #before 23:00 31/12/2017 (2014-2017) 
    print(np.shape(training))
    testing = df[df['Index']>=35065]#1/1/2018- 31/12/2018
    print(np.shape(testing))
    
    head, tail = os.path.split(filename)
    tail2 = tail.split('.')[0]
    train_name = dest_dir+tail2+'_train.csv'
    test_name = dest_dir+tail2+'_test.csv'
    print(train_name)
    print(test_name)
    training.to_csv(train_name,index=False)
    testing.to_csv(test_name,index=False)

    
for filename in filenames:
    print(filename)
    data = pd.read_csv(filename)
    del data["DateTime"]
    data = FillMissingValues(data)
    print(np.shape(data))
    Morning_Data = data[data.Hours>=6]
    Morning_Data = Morning_Data[Morning_Data.Hours<=10]
    Evening_Data = data[data.Hours>=17]
    Evening_Data = Evening_Data[Evening_Data.Hours<=21]
    TargetName = data.columns[-1]
    #get maximum values
    size=5 #6-10 hours
    dfs = split_dataframe_to_chunks(Morning_Data,size)
    Max_df_Morning = Get_MaxAndARandomValue(dfs,TargetName)
    #17:00-21:00
    size= 5
    dfs = split_dataframe_to_chunks(Evening_Data,size)
    Max_df_Evening = Get_MaxAndARandomValue(dfs,TargetName)
    #Fullday 0-23:00
    size= 24
    dfs = split_dataframe_to_chunks(data,size)
    Max_df_fullDay = Get_MaxAndARandomValue(dfs,TargetName)
    
    #delete column TargetName.
    Max_df_Morning = Max_df_Morning.drop(TargetName,axis=1)
    Max_df_Evening = Max_df_Evening.drop(TargetName,axis=1)
    Max_df_fullDay = Max_df_fullDay.drop(TargetName,axis=1)
    
    Write_To_File(dest_dir_Morning,Max_df_Morning, filename)
    Write_To_File(dest_dir_Evening,Max_df_Evening, filename)
    Write_To_File(dest_dir_FullDay,Max_df_fullDay, filename)

print('DONE')

C:\data\PeakConcentration-Idea2-2-Clasification\data\CO.csv
0
0
1611
1611
(43824, 12)
(3622, 12)
(2896, 12)
(726, 12)
C:\data\PeakConcentration-Idea2-2-Clasification\Morning_Peak_Values\CO_train.csv
C:\data\PeakConcentration-Idea2-2-Clasification\Morning_Peak_Values\CO_test.csv
(3626, 12)
(2900, 12)
(726, 12)
C:\data\PeakConcentration-Idea2-2-Clasification\Evening_Peak_Values\CO_train.csv
C:\data\PeakConcentration-Idea2-2-Clasification\Evening_Peak_Values\CO_test.csv
(3636, 12)
(2906, 12)
(730, 12)
C:\data\PeakConcentration-Idea2-2-Clasification\FullDay_Peak_Values\CO_train.csv
C:\data\PeakConcentration-Idea2-2-Clasification\FullDay_Peak_Values\CO_test.csv
C:\data\PeakConcentration-Idea2-2-Clasification\data\NO2.csv
0
0
1455
1455
(43824, 12)
(3605, 12)
(2880, 12)
(725, 12)
C:\data\PeakConcentration-Idea2-2-Clasification\Morning_Peak_Values\NO2_train.csv
C:\data\PeakConcentration-Idea2-2-Clasification\Morning_Peak_Values\NO2_test.csv
(3615, 12)
(2889, 12)
(726, 12)
C:\data\PeakConcentra